# Problem Statement

## Q1. Linear Programming

Your start-up will face the cash requirements shown in Table 1 in the next eight quarters (positive entries represent cash needs while negative entries represent cash surpluses). The company has three borrowing possibilities.

**Table 1: Cash Flow (in Crores of INR)**

| Q1  | Q2  | Q3  | Q4   | Q5   | Q6  | Q7  | Q8   |
| --- | --- | --- | ---- | ---- | --- | --- | ---- |
| 100 | 500 | 100 | -600 | -500 | 200 | 600 | -900 |

**Borrowing Possibilities:**

*   A 2-year loan available at the beginning of Q1, with a 1% interest per quarter.
*   The other two borrowing opportunities are available at the beginning of every quarter:
    *   A 6-month loan with a 1.8% interest per quarter.
    *   A quarterly loan with a 2.5% interest for the quarter.
*   Any surplus can be invested at a 0.5% interest per quarter.

**Questions:**

a) Formulate a LP that maximizes the wealth of the company at the beginning of Q9.

b) Use built-in LP solvers to get a solution.

c) What is a good initial basic feasible point?

d) Use the revised simplex method and see if your solution matches the one from built-in solver. Explain.

# My approach
## Assumptions
- <span style="color:blue">We pay compound interest with the principal at the end of the loan term</span>
- <span style="color:blue">We cannot take a 6 month loan at the beginning of the 8th quarter as this will be repayed after the ninth quarter</span>
- <spam style="color:red">If the cashflow is negative for a quarter it cannot be invested in the same quarter</span>

## Variables

- $x_t$: Total money at the start of quarter $t$ after all loans have also been taken.  
- $L$: Two-year loan available at the beginning of the first quarter
- $i_t$: The amount invested in quarter $t$.
- $j_t$: 6-month loan available at the beginning of each quarter $t$  
- $k_t$: 4-month loan available at the beginning of each quarter $t$    
- $c_t$: Cash flow in quarter $t$ provided in the table (Technically $c_t$ is not a decision variable as it's values are provided) 
- $r$: Final result (e.g., total return) to be maximized


## Constraints

### General constraints

- $x_t \geq 0$ , $i_t \geq0$,$x_t>=i_t$,  $x_t-i_t \geq c_t$,  $j_t \geq 0$, $k_t \geq 0$,  $L \geq 0$

All $c_t$ values must match the given values


### Quarter 1 constraints
- $x_1 = L + j_1 + k_1 $
- $x_2 = x_1 - c_1 + 0.005 \cdot (i_1) - 1.025 \cdot k_1$

*(Note: Expressions will change if banks demand that previous loans be repaid before new loans etc.)*

### General constraints (for $t = 2$ to $7$)
- $x_{t+1} = x_t - c_t  +0.005 \cdot i_t - 1.025 \cdot k_t - (1+0.018)^2 \cdot j_{t-1} + j_{t+1} + k_{t+1}$

### Final quarter constraints
- $j_8 = 0$ No 6 month loan in final quarter 
### Objective
- $r = x_9 - (1+0.01)^8 \cdot L$  
  *(Maximize $r$, the net return after repaying the two-year loan) from the amount at the start of quarter 9*


In [1]:
import numpy as np
import pulp

# Create the LP problem
lpp = pulp.LpProblem("Cash_Flow_Management", pulp.LpMaximize)

# Define the given cash flow requirements
cash_flow = [None,100, 500, 100, -600, -500, 200, 600, -900]
quarters = range(1, 9)  # Quarters 1 through 8

# Define variables
L = pulp.LpVariable("L", lowBound=0)  # 2-year loan available at beginning of Q1
x = {t: pulp.LpVariable(f"x_{t}", lowBound=0) for t in range(1, 10)}  # Total money at start of quarter t
i = {t: pulp.LpVariable(f"i_{t}", lowBound=0) for t in quarters}  # Amount invested in quarter t
j = {t: pulp.LpVariable(f"j_{t}", lowBound=0) for t in quarters}  # 6-month loan at beginning of quarter t
k = {t: pulp.LpVariable(f"k_{t}", lowBound=0) for t in quarters}  # Quarterly loan at beginning of quarter t
r = pulp.LpVariable("r")  # Final return to be maximized

# Objective function: Maximize wealth at beginning of Q9
lpp += r, "Maximize_Wealth"

# Constraint for r (final return)
lpp += r == x[9] - (1 + 0.01) ** 8 * L, "Final_Return_Constraint"

# Quarter 1 constraints
lpp += x[1] == L + j[1] + k[1], "Q1_Money_Balance"
lpp += x[2] == x[1] - cash_flow[1] + 0.005 * i[1] - 1.025 * k[1] + j[2] + k[2], "Q1_to_Q2_Balance"

# Constraints for quarters 2 to 7
for t in range(2, 8):
    lpp += x[t+1] == x[t] - cash_flow[t] + 0.005 * i[t] - 1.025 * k[t] - (1 + 0.018)**2 * j[t-1] + j[t+1] + k[t+1], f"Q{t}_to_Q{t+1}_Balance"

# Constraint for quarter 8
lpp += x[9] == x[8] - cash_flow[8] + 0.005 * i[8] - 1.025 * k[8] - (1 + 0.018)**2 * j[7], "Q8_to_Q9_Balance"

# No 6-month loan in final quarter
lpp += j[8] == 0, "No_6month_Loan_Q8"

# Investment constraints: can't invest more than what we have after meeting cash requirements
for t in quarters:
    if cash_flow[t] >=0:
        lpp += x[t] - i[t] >= cash_flow[t], f"Cash_Flow_Requirement_Q{t}"
    else:
        lpp += i[t] <= x[t], f"Investment_Limit_Q{t}"

# Solve the problem
lpp.solve()

# Print the status of the solution
print("Status:", pulp.LpStatus[lpp.status])

# Print the optimal values of the variables
print("\nOptimal Solution:")
print(f"2-Year Loan (L): {pulp.value(L):.2f}")
for t in quarters:
    print(f"\nQuarter {t}:")
    print(f"Total Money (x_{t}): {pulp.value(x[t]):.2f}")
    print(f"Investment (i_{t}): {pulp.value(i[t]):.2f}")
    print(f"6-Month Loan (j_{t}): {pulp.value(j[t]):.2f}")
    print(f"Quarterly Loan (k_{t}): {pulp.value(k[t]):.2f}")
print(f"\nTotal Money at Q9 (x_9): {pulp.value(x[9]):.2f}")
print(f"Final Return (r): {pulp.value(r):.2f}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/harshit/anaconda3/envs/ml/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/e24cd85777f647879acd7c46a589ce22-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/e24cd85777f647879acd7c46a589ce22-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 24 COLUMNS
At line 103 RHS
At line 123 BOUNDS
At line 125 ENDATA
Problem MODEL has 19 rows, 35 columns and 77 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 13 (-6) rows, 29 (-6) columns and 69 (-8) elements
Perturbing problem by 0.001% of 3.8464207 - largest nonzero change 4.6390231e-05 ( 0.0082892611%) - largest zero change 4.510326e-05
0  Obj -0.034716429 Primal inf 28177.196 (9) Dual inf 3.8464183 (1)
0  Obj -0 Primal inf 28177.196 (9) Dual inf 7.8880719e+11 (14)
18  Obj 457.85684
Optimal - objective value 457.85684
After P

# Results
- Final return <span style="color:green">457.85</span>
- Allowing loan a six month loan at the start of quarter 8 will result in an unbounded solution. <span style="color:blue">If we explicitly substract the amount to be repaid then we get the optimal solution does not take any loan</span>
- If we also assume the surplus is available at the start of the quarter then we need to get comment line 41 <span style="color:green"> 467.95 </span> is obtained.